## Baba is You

You are usually controlling baba, a sheep. You are allowed to do anything specified by the rules, and you win if there is a win rule and you reach the win object specified by the win rule.

For instance, in the following game set-up you are baba, as defined by the rule 'baba is you', the wall is stop (meaning you can't move it) and nothing is currently set to the win object, but it could be if you move the text objects around to create it. Everything is moveable except for the wall objects.  

<img src="babaExample.png" style="width: 700px; height: 500px;">

In [109]:
import numpy as np
import matplotlib.pyplot as plt
import copy

Pro tip: alt + enter exits full screen on steam

### Stages

Caps represent text, lowercase is object

In [279]:
stage2 = np.array([['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','flag','flag','flag','flag','flag','flag','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','0','is','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','flag','flag','flag','flag','flag','0','0','0','0','win','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0'],
['0','0','0','0','0','0','0','0','0','0','flag','0','baba','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','flag','flag','flag','flag','flag','flag','flag','flag','flag','flag','flag','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','0','0','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','WALL','0','0','flag','0','FLAG','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','IS','0','0','flag','0','IS','0','0','wall','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','YOU','0','0','flag','0','STOP','0','0','0','0','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','0','0','0','flag','flag','flag','flag','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','flag','flag','flag','flag','flag','0','0','0','0','0','0','0','0','0','0','0'],\
['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0']])


### Parameters and static objects

Directions: 0 is up, 1 is right, 2 is down, 3 is left

In [385]:
allObjectText = ['WALL','FLAG','ROCK','BABA']
allCommandText = ['YOU', 'STOP','WIN']
isText = ['IS']
allText = allObjectText + allCommandText + isText
allObjects = ['flag', 'wall', 'baba','rock']
allItems = allText + allObjects
reward = 10

### Updating the board

Fixes and tests to update board:
- The function will currently not work correctly if one agent is stacked on another (in some cases). It should update the board so that they both move up one space, but right now it will do that only if you iterate through the agents in the right way, but it might not if you start with the bottom one. Instead it would push them both up by one then when it gets to the top one, also move that up by one so they'd be spaced one apart
    I think one way to fix is this is in the updateBoard function -- make sure you go through agents in order
- Currently the only agents this considers are the 'yous', I need to introduce other types of agents like 'MOVE' and 'keke'

In [11]:
def coordinates(whereArray):
    """returns x,y coordinates given a numpy object"""
    array = []
    for i in range(len(whereArray[0])):
        array.append([whereArray[0][i],whereArray[1][i]])
    return array

In [12]:
def findObject(obj, board):
    """returns a list of coordinates given an object name"""
    objects = np.where(board==obj)
    return coordinates(objects)

In [124]:
def findYou(board):
    """returns the coordinates of where the agents are"""
    # need to figure out what to do if there are no yous, or if an agent isn't you (like a keese?) -- will keep updating as I go through the levels
    YOU = np.where(board=='YOU')
    YOU = coordinates(YOU)
    yous = []
    you_coords = []

    for you in YOU:
    # determining if the top represents a you
        if board[you[0]-2][you[1]] in allObjectText and board[you[0]-1][you[1]] == 'IS':
            yous.append(board[you[0]-2][you[1]])
        # determining if the left hand side represents a you
        elif board[you[0]][you[1]-2] in allObjectText and board[you[0]][you[1]-1] == 'IS':
            yous.append(board[you[0]-2][you[1]])
    for you in yous:
        for objs in findObject(you.lower(),board):
            you_coords.append(objs)
            
    return you_coords

In [53]:
def rules(board):
    """returns a list of lists, each list is a list of strings which represent a rule"""
    IS = np.where(board=='IS')
    IS = coordinates(IS)
    rules = []

    for rule in IS:
    # determing if the top is a rule
        if board[rule[0]-1][rule[1]] in allObjectText and board[rule[0]+1][rule[1]] in allCommandText:
            rules.append([board[rule[0]-1][rule[1]],'IS',board[rule[0]+1][rule[1]]])
        # determining if the left hand side represents a IS
        elif board[rule[0]][rule[1]-1] in allObjectText and board[rule[0]][rule[1]+1] in allCommandText:
            rules.append([board[rule[0]][rule[1]-1],'IS',board[rule[0]][rule[1]+1]])
    return rules

In [286]:
def unMoveables(board):
    """returns a list of all the unmoveable objects on the grid, i.e., those that are stopped"""
    objs = []
    for rule in rules(board):
        if 'STOP' in rule:
            objs.append(rule[0].lower())
    return objs

In [337]:
def gridDim(board):
    """returns height, and width"""
    return len(board[:][0]) - 1, len(board[0][:]) - 1

In [338]:
def step(board, agent, direction):
    """given an agent, board, and direction, returns a move given that there is nothing blocking you"""
    height, width = gridDim(board) 
    if direction == 0:
        if agent[0] == 0:
            return None
        return [agent[0]-1, agent[1]]
    if direction == 1:
        if agent[1] == width:
            return None
        return [agent[0], agent[1]+1]
    if direction == 2:
        if agent[0] == height:
            return None
        return [agent[0]+1, agent[1]]
    if direction == 3:
        if agent[1] == 0:
            return None
        return [agent[0],agent[1]-1]

In [253]:
def moveNoObjects(board, agent_coords, move_coords, agent_text):
    """returns an updated board given no objects in the direction of movement"""
    new_board = copy.deepcopy(board)
    new_board[move_coords[0]][move_coords[1]] = agent_text
    new_board[agent_coords[0]][agent_coords[1]] = str(0)
    return new_board

In [387]:
def moveObjects(board, agent_coords, direction):
    """returns an updated board given objects in the direction of movement"""
    new_board = copy.deepcopy(board)
    obstacles = []
    height, width = gridDim(board)
    
    if direction == 0:
        r = range(agent_coords[0])
        obs = [[agent_coords[0]-(i+1),agent_coords[1]] for i in r]
        old_coords = [[agent_coords[0]-i,agent_coords[1]] for i in r]
    if direction == 1:
        r = range(width - agent_coords[1])
        obs = [[agent_coords[0],agent_coords[1]+(i+1)] for i in r]
        old_coords = [[agent_coords[0],agent_coords[1]+i] for i in r]
    if direction == 2:
        r = range(height - agent_coords[0])
        obs = [[agent_coords[0]+(i+1),agent_coords[1]] for i in r]
        old_coords = [[agent_coords[0]+i,agent_coords[1]] for i in r]
    if direction == 3:
        r = range(agent_coords[1])
        obs = [[agent_coords[0],agent_coords[1]-(i+1)] for i in r]
        old_coords = [[agent_coords[0],agent_coords[1]-i] for i in r]
    # check if they won, i.e., the object immediately next to them is a win object
    if win(board,[obs[0][0],obs[0][1]]):
        return reward
    # creating a list of objects next to the agent
    for ob in obs:
        if board[ob[0]][ob[1]] in allItems:
            obstacles.append(board[ob[0]][ob[1]])
        # if the item above you is empty space, terminate the for loop
        if board[ob[0]][ob[1]] == str(0):
            break
        # if objects stack to the edge of the grid, moving won't do anything
        if (ob[0] == 0 and direction == 0) or (ob[1] == width and direction == 1) or \
        (ob[0]== height and direction == 2) or (ob[1] == 0 and direction == 3):
            return new_board
        
    # if there's a match between the items above you and unmoveable objects, you don't move
    if set(obstacles) & set(unMoveables(board)):
        return new_board
    
    # if there isn't, update the board accordingly
    else:
        updateCoords = obs[:len(obstacles)+1]
        oldCoords = old_coords[:len(obstacles)+1]
        for ob in zip(updateCoords,oldCoords):
            new, old = ob[0], ob[1]
            new_board[new[0]][new[1]] = board[old[0]][old[1]]
        new_board[agent_coords[0]][agent_coords[1]] = str(0)
    return new_board

In [321]:
def move(board, agent, direction):
    """updates the board to reflect a single agents movement (or nonmovement)"""
    agent_text = board[agent[0]][agent[1]]
    move_coords = step(board, agent, direction)
    
    # if you're at the edge you can't move
    if move_coords is None:
        return board
    #and if there isn't anything beside you, you move one space
    if board[move_coords[0]][move_coords[1]] == str(0):
        new_board = moveNoObjects(board, agent, move_coords, agent_text)
    # otherwise, you move objects when you move
    else:
        new_board = moveObjects(board, agent, direction)
        
    return new_board

In [349]:
def agentOrder(agents, direction):
    """ given a list of agent coordinates, returns a sorted list with the ones closest to the edges first """
    if direction == 0:
        return sorted(agents, key=lambda x: x[0])
    if direction == 1:
        return sorted(agents, key=lambda x: x[1], reverse=True)
    if direction == 2:
        return sorted(agents, key=lambda x: x[0], reverse=True)
    if direction == 3:
        return sorted(agents, key=lambda x: x[1])

In [352]:
def updateBoard(board, direction):
    """update board based on direction provided. Returns a numpy array"""
    assert direction in {0,1,2,3}
    new_board = copy.deepcopy(board)
    agents = agentOrder(findYou(board), direction)
    
    " moving up "
    if direction == 0:
        for agent in agents:
            new_board = move(new_board, agent, 0)
        return new_board
    
    " moving right "
    if direction == 1:
        for agent in agents:
            new_board = move(new_board, agent, 1)
        return new_board
    
    " moving down "
    if direction == 2:
        for agent in agents:
            new_board = move(new_board, agent, 2)
        return new_board
    
    " moving left "
    if direction == 3:
        for agent in agents:
            new_board = move(new_board, agent, 3)
        return new_board

In [388]:
def win(board, agent_coords):
    """given a board and coordinates, win returns true if the board at those coordinates is a winning object
    and false otherwise"""
    win = None
    for rule in rules(board):
        if 'WIN' in rule: 
            win = rule[0].lower()
    if win != None:
        if board[agent_coords[0]][agent_coords[1]] == win:
            return True
        else:
            return False
    return False

In [366]:
fakeStage = np.array([['BABA','0','0'],['IS','baba','0'],['YOU','0','0']])
fakeStage2 = updateBoard(fakeStage,0)
print(fakeStage)
print(fakeStage2)

[['BABA' '0' '0']
 ['IS' 'baba' '0']
 ['YOU' '0' '0']]
[['BABA' 'baba' '0']
 ['IS' '0' '0']
 ['YOU' '0' '0']]


In [367]:
fakeStage3 = updateBoard(fakeStage2,0)
print(fakeStage3)

[['BABA' 'baba' '0']
 ['IS' '0' '0']
 ['YOU' '0' '0']]


In [368]:
fakeStage2 = updateBoard(fakeStage,1)
print(fakeStage2)

[['BABA' '0' '0']
 ['IS' '0' 'baba']
 ['YOU' '0' '0']]


In [379]:
bigStage = np.array([['BABA','ROCK','0','0','0'],['IS', 'IS', 'rock','0','0'],['YOU','WIN','rock','rock','rock'],\
                    ['0','0','baba','0','0'],['0','0','flag','0','0']])

In [380]:
print(bigStage)

[['BABA' 'ROCK' '0' '0' '0']
 ['IS' 'IS' 'rock' '0' '0']
 ['YOU' 'WIN' 'rock' 'rock' 'rock']
 ['0' '0' 'baba' '0' '0']
 ['0' '0' 'flag' '0' '0']]


In [389]:
bigStage2 = updateBoard(bigStage, 0)
print(bigStage2)

10


In [372]:
bigStage3 = updateBoard(bigStage2,2)
print(bigStage3)

[['BABA' 'ROCK' '0' '0' '0']
 ['IS' 'IS' 'rock' '0' '0']
 ['YOU' 'STOP' 'rock' 'rock' 'rock']
 ['0' '0' 'baba' '0' '0']
 ['0' '0' 'flag' '0' '0']]


In [373]:
print(updateBoard(bigStage3,2))

[['BABA' 'ROCK' '0' '0' '0']
 ['IS' 'IS' 'rock' '0' '0']
 ['YOU' 'STOP' 'rock' 'rock' 'rock']
 ['0' '0' 'baba' '0' '0']
 ['0' '0' 'flag' '0' '0']]


In [374]:
print(rules(stage2))

[['WALL', 'IS', 'YOU'], ['FLAG', 'IS', 'STOP']]
